In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

gpu = tf.test.gpu_device_name()
gpu

from tensorflow.python.client import device_lib
device_lib.list_local_devices()



In [ ]:
!pip install wandb==0.12.2

!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip
!unzip nature_12K.zip
!rm nature_12K.zip

!mv ./inaturalist_12K/val ./inaturalist_12K/test

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os, sys
sys.path.append("/content/drive/MyDrive/CS6910/Assignment2")


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2 as IRV2
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.xception import Xception


from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, InputLayer, Flatten, Conv2D, BatchNormalization, MaxPooling2D, Activation , GlobalAveragePooling2D
from tensorflow.keras.models import Sequential,  Model

import wandb


class ObjectDetection():

    def __init__(self, image_size, modelConfigDict, using_pretrained_model = False, base_model = "IRV2" ):
        
        self.no_hidden_cnn_layers= modelConfigDict["no_hidden_cnn_layers"]
        self.activation = modelConfigDict["activation"]
        self.batch_norm = modelConfigDict["batch_norm"]
        self.filter_dist = modelConfigDict["filter_dist"]
        self.filter_size = modelConfigDict["filter_size"]
        self.no_of_filters_base  = modelConfigDict["no_of_filters_base"]
        self.dropout_fraction = modelConfigDict["dropout_fraction"]
        self.pool_size = modelConfigDict["pool_size"]
        self.padding = modelConfigDict["padding"]
        self.dense_neurons = modelConfigDict["dense_neurons"]
        self.num_classes = modelConfigDict["num_classes"]
        self.optimizer = modelConfigDict["optimizer"]
        self.global_avg_pooling = modelConfigDict["global_avg_pooling"]
        self.batch_normalisation_location = modelConfigDict["batch_normalisation_location"]
        BASE_MODELS = {
        "IRV2" : IRV2,
        "IV3" : InceptionV3,
        "RN50" : ResNet50,
        "XCPTN" : Xception
        }      
        
        if using_pretrained_model == True:
            self.base_model = base_model

            if self.base_model == "RN50":
                self.IMG_HEIGHT = 224
                self.IMG_WIDTH = 224

            else:
                self.IMG_HEIGHT = image_size[0]
                self.IMG_WIDTH = image_size[1]        

        self.IMG_HEIGHT = image_size[0]
        self.IMG_WIDTH = image_size[1]        
         
        self.input_shape = (self.IMG_HEIGHT, self.IMG_WIDTH, 3)

    def build_cnnmodel_dense(self):

        with tf.device('/device:GPU:0'):
            tf.keras.backend.clear_session()
            model = Sequential()
            
            #Input Layer
            model.add(Conv2D(self.no_of_filters_base, self.filter_size ,kernel_initializer = "he_uniform",padding = self.padding,input_shape = (self.IMG_HEIGHT, self.IMG_WIDTH, 3)))
            if self.batch_normalisation_location == "Before" and self.batch_norm: model.add(BatchNormalization())
            model.add(Activation(self.activation))
            
            #Performing Batch_normalisation
            if self.batch_normalisation_location == "After" and self.batch_norm: model.add(BatchNormalization())
            
            #Max pooling
            model.add(MaxPooling2D(pool_size=self.pool_size))  
            for i in range(self.no_hidden_cnn_layers-1):
            
                ## Standard filter 
                if self.filter_dist == "standard":
                    model.add(Conv2D(self.no_of_filters_base, self.filter_size,kernel_initializer = "he_uniform",padding = self.padding))
            
                ## Double filter 
                elif self.filter_dist == "double":
                    model.add(Conv2D(2**(i+1)*self.no_of_filters_base, self.filter_size,kernel_initializer = "he_uniform",padding = self.padding))
            
                ## Half the filter size in each of the convolutional layers
                elif self.filter_dist == "half":
                    model.add(Conv2D(int(self.no_of_filters_base/2**(i+1)),self.filter_size, kernel_initializer = "he_uniform"))
            
                if self.batch_normalisation_location == "Before" and self.batch_norm: model.add(BatchNormalization())
                model.add(Activation(self.activation))
            
                if self.batch_normalisation_location == "After" and self.batch_norm: model.add(BatchNormalization())
                
                model.add(MaxPooling2D(pool_size=self.pool_size))
            
            #Densely connected layers
            if self.global_avg_pooling == True:
                model.add(GlobalAveragePooling2D())
            else:
                model.add(Flatten())
            model.add(Dense(self.dense_neurons, activation = 'sigmoid'))
            if self.dropout_fraction != None:
                model.add(tf.keras.layers.Dropout(self.dropout_fraction))
            model.add(Dense(self.num_classes, activation = 'softmax'))
          
            return model 

    def build_cnnmodel_conv(self):
        with tf.device('/device:GPU:0'):
            tf.keras.backend.clear_session()
            model = Sequential()
            
            #First CNN layer connecting to input layer
            model.add(Conv2D(self.no_of_filters_base, self.filter_size, padding = self.padding,kernel_initializer = "he_uniform", input_shape = (self.IMG_HEIGHT, self.IMG_WIDTH, 3)))
            if self.batch_normalisation_location == "Before" and self.batch_norm: model.add(BatchNormalization())
            model.add(Activation(self.activation))
            
            #batch_normalisation
            if self.batch_normalisation_location == "After" and self.batch_norm: model.add(BatchNormalization())
            #max pooling
            model.add(MaxPooling2D(pool_size=self.pool_size))  
            if self.dropout_fraction != None:
                model.add(tf.keras.layers.Dropout(self.dropout_fraction))
            for i in range(self.no_hidden_cnn_layers-1):
                #i+2th Convolutional Layer
            
                ## Standard filter distribution - same number of filters in all Convolutional layers
                if self.filter_dist == "standard":
                    model.add(Conv2D(self.no_of_filters_base, self.filter_size,kernel_initializer = "he_uniform",padding = self.padding))
            
                ## Double filter distribution - double number of filters in each Convolutional layers
                elif self.filter_dist == "double":
                    model.add(Conv2D(2**(i+1)*self.no_of_filters_base, self.filter_size,kernel_initializer = "he_uniform", padding = self.padding))
            
                ## Halve the filter size in each successive convolutional layers
                elif self.filter_dist == "half":
                    model.add(Conv2D(int(self.no_of_filters_base/2**(i+1)), self.filter_size,kernel_initializer = "he_uniform", padding = self.padding))
            
                if self.batch_normalisation_location == "Before" and self.batch_norm: model.add(BatchNormalization())
                model.add(Activation(self.activation))
            
                if self.batch_normalisation_location == "After" and self.batch_norm: model.add(BatchNormalization())
            
                model.add(MaxPooling2D(pool_size=self.pool_size))
                if self.dropout_fraction != None:
                    model.add(tf.keras.layers.Dropout(self.dropout_fraction))
            
            #Final densely connected layers
            if self.global_avg_pooling == True:
                model.add(GlobalAveragePooling2D())
            else:
                model.add(Flatten())

            model.add(Dense(self.dense_neurons, activation = 'sigmoid'))
            model.add(Dense(self.num_classes, activation = 'softmax'))

            return model      
        
    def build_cnnmodel_all(self):

        with tf.device('/device:GPU:0'):    
            tf.keras.backend.clear_session()
            model = Sequential()
            
            #First CNN layer
            model.add(Conv2D(self.no_of_filters_base, self.filter_size, padding = self.padding,kernel_initializer = "he_uniform", input_shape = (self.IMG_HEIGHT, self.IMG_WIDTH, 3)))
            model.add(Activation(self.activation))
            
            #Batch_normalisation
            if self.batch_normalisation_location == "After" and self.batch_norm: model.add(BatchNormalization())
            
            #Max pooling
            model.add(MaxPooling2D(pool_size=self.pool_size))  
            if self.dropout_fraction != None:
                model.add(tf.keras.layers.Dropout(self.dropout_fraction))

            for i in range(self.no_hidden_cnn_layers-1):
            
                ## Standard filter distribution
                if self.filter_dist == "standard":
                    model.add(Conv2D(self.no_of_filters_base, self.filter_size,kernel_initializer = "he_uniform", padding = self.padding))
            
                ## Double filter distribution
                elif self.filter_dist == "double":
                    model.add(Conv2D(2**(i+1)*self.no_of_filters_base, self.filter_size,kernel_initializer = "he_uniform",padding = self.padding))
            
                ## Halve the filter size in each successive convolutional layers
                elif self.filter_dist == "half":
                    model.add(Conv2D(int(self.no_of_filters_base/2**(i+1)), self.filter_size,kernel_initializer = "he_uniform", padding = self.padding))
            
                model.add(Activation(self.activation))
            
                if self.batch_normalisation_location == "After" and self.batch_norm: model.add(BatchNormalization())
            
                model.add(MaxPooling2D(pool_size=self.pool_size))
                if self.dropout_fraction != None:
                    model.add(tf.keras.layers.Dropout(self.dropout_fraction))
            
            #Densely connected layers
            if self.global_avg_pooling == True:
                model.add(GlobalAveragePooling2D())
            else:
                model.add(Flatten())

            model.add(Dense(self.dense_neurons, activation = 'sigmoid'))
            if self.dropout_fraction != None:
                model.add(tf.keras.layers.Dropout(self.dropout_fraction))
            model.add(Dense(self.num_classes, activation = 'softmax'))
            
            return model      
      
    
                 
        
    def load_pretrained_model(self):
        
        base_model = BASE_MODELS[self.base_model_name]
        base = base_model(weights='imagenet', include_top=False)
        x = base.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(self.dense_neurons, activation='relu')(x)
        guess = Dense(self.num_classes, activation='softmax')(x)
        model = Model(inputs=base.input, outputs=guess)

        # freeze the base layers
        for l in base.layers:
            l.trainable = False

        return model
 

In [ ]:
#Preprocessing the data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pathlib


import wandb
from wandb.keras import WandbCallback

IMG_SIZE = (128,128)

'''
#sweep config
sweep_config = {
  "name": "Bayesian Sweep",
  "method": "bayes",
  "metric":{
  "name": "val_accuracy",
  "goal": "maximize"
  },
  'early_terminate': {
        'type':'hyperband',
        'min_iter': [3],
        's': [2]
  },
  "parameters": {
        
        "activation":{
            "values": ["relu", "elu", "selu"]
        },
        "filter_size": {
            "values": [(2,2), (3,3), (4,4)]
        },
        "batch_size": {
            "values": [32, 64]
        },
        "padding": {
            "values": ["same","valid"]
        },
        "data_augmentation": {
            "values": [True, False]
        },
        "optimizer": {
            "values": ["sgd", "adam", "rmsprop", "nadam"]
        },
        "batch_normalization": {
            "values": [True, False]
        },
        "batch_normalisation_location": {
            "values": ["Before", "After"]
        },
        "number_of_filters_base": {
            "values": [32, 64]
        },
        "dense_neurons": {
            "values": [32, 64, 128]
        },   
        "dropout_location": {
            "values": ["conv","dense","all"]
        },
        "dropout_fraction": {
            "values": [None, 0.2,0.3]
        },  
        "global_average_pooling": {
            "values": [False,True]
        },        
    }
}

sweep_id = wandb.sweep(sweep_config,project='Assignment-2', entity='karanwxlia')
'''


#train function
def train():

        
    config_defaults = dict(
            num_hidden_cnn_layers = 5 ,
            activation = 'relu',
            batch_normalization = True,
            batch_normalisation_location = "After",
            filter_distribution = "double" ,
            filter_size = (3,3),
            number_of_filters_base  = 32,
            dropout_fraction = None,
            dropout_location = "dense",
            pool_size = (2,2),
            padding = 'same',
            dense_neurons = 128,
            num_classes = 10,
            optimizer = 'adam',
            epochs = 5,
            batch_size = 32, 
            data_augmentation = False,
            global_average_pooling = True,
            img_size = IMG_SIZE
        ) 



    wandb.init(project = 'Assignment-2', config = config_defaults,entity='karanwxlia')
    CONFIG = wandb.config

    #RUN name for wandb
    wandb.run.name = "OBJDET_" + str(CONFIG.num_hidden_cnn_layers) + "_dn_" + str(CONFIG.dense_neurons) + "_opt_" + CONFIG.optimizer + "_dro_" + str(CONFIG.dropout_fraction) + "_bs_"+str(CONFIG.batch_size) + "_fd_" + CONFIG.filter_distribution + "_bnl_" + CONFIG.batch_normalisation_location + "_dpl_" + CONFIG.dropout_location
    data_augmentation = CONFIG.data_augmentation
    BATCH_SIZE = CONFIG.batch_size

    if data_augmentation == True:

    #Better Alternative
        train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255,
                val_split = 0.1,
                shear_range=0.2,
                zoom_range=0.2,
                featurewise_center=False,  
                samplewise_center=False,  
                featurewise_std_normalization=False,
                samplewise_std_normalization=False, 
                zca_whitening=False,  
                rotation_range=15,  
                width_shift_range=0.1,
                height_shift_range=0.1,
                horizontal_flip=True,  
                vertical_flip=False
                )
    else:
        train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,val_split = 0.1)

    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


    train_generator = train_datagen.flow_from_directory(
        './inaturalist_12K/train',
        subset='training',
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle = True,
        seed = 123)
        
    validation_generator = train_datagen.flow_from_directory(
            './inaturalist_12K/train',
            target_size=IMG_SIZE,
            subset = 'validation',
            batch_size=BATCH_SIZE,
            class_mode='categorical',
            shuffle = True,
            seed = 123)


            
    test_generator = test_datagen.flow_from_directory(
            './inaturalist_12K/test',
            target_size=IMG_SIZE,
            batch_size=BATCH_SIZE,
            class_mode='categorical',
            shuffle = True,
            seed = 123)



    with tf.device('/device:GPU:0'):        
        objDetn = ObjectDetection(CONFIG.img_size, CONFIG )
        if CONFIG.dropout_location == "all":
            model = objDetn.build_cnnmodel_all()
        elif CONFIG.dropout_location == "conv":
            model = objDetn.build_cnnmodel_conv()
        elif CONFIG.dropout_location == "dense":
            model = objDetn.build_cnnmodel_dense()
        
        model.summary()



        model.compile(
        optimizer=CONFIG.optimizer,  
        # Loss function 
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'],
        )
      
        history = model.fit(
                        train_generator,
                        steps_per_epoch = train_generator.samples // CONFIG.batch_size,
                        validation_data = validation_generator, 
                        validation_steps = validation_generator.samples // CONFIG.batch_size,
                        epochs = CONFIG.epochs, 
                        callbacks=[WandbCallback()]
                        )

        model.save('./TrainedModel/'+wandb.run.name)
        wandb.finish()
        return model, history
        


In [ ]:
train()

In [ ]:
#wandb.agent(sweep_id, train, count = 100)

In [ ]:
!cp -rf /content/TrainedModel/OBJDET_5_dn_128_opt_nadam_dro_None_bs_32_fd_double_bnl_After_dpl_all  /content/drive/MyDrive/CS6910/Assignment2/Best_trained_Model